In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np

### Download

In [8]:
# Загрузим данные, изучим их

train = pd.read_csv('data/train.csv')
print(train.shape)
train.head()

(38740, 3)


,oid,category,text
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...


In [9]:
test = pd.read_csv('data/test.csv')
print(test.shape)
test.head()

(26260, 2)


,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...


In [10]:
sample = pd.read_csv('data/sample_submission.csv')
print(sample.shape)
sample.head()

(2626, 2)


,oid,category
0,1622114,athletics
1,1663817,autosport
2,3174332,basketball
3,3469228,extreme
4,3905302,boardgames


In [11]:
print(sample['oid'].value_counts())
print(test['oid'].value_counts())

oid
999374432    1
1622114      1
995180722    1
995078418    1
994379631    1
            ..
4029918      1
3914130      1
3905302      1
3469228      1
3174332      1
Name: count, Length: 2626, dtype: int64
oid
952958325    10
749208109    10
300872233    10
770007815    10
837661534    10
             ..
306441436    10
332983824    10
566255305    10
663621910    10
161038103    10
Name: count, Length: 2626, dtype: int64


In [12]:
# Объеденим test и sample по iod, чтобы у тестовой выборки была размечена целевая переменная

test = pd.merge(test, sample, on='oid', how='inner')
test.head()

,oid,text,category
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...,esport
1,452466036,Естественное восстановление после тяжелой трен...,martial_arts
2,161038103,Тема нарядов продолжается Одна из британских ж...,tennis
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...,esport
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...,tennis


### Preparation

In [13]:
# Удалим столбец с id, после проверим на дубликаты

train = train.drop(columns=['oid'])
test = test.drop(columns=['oid'])

In [14]:
print(f'Количество дубликатов в train: {train.duplicated().sum()}')
print(f'Количество дубликатов в test: {test.duplicated().sum()}')

Количество дубликатов в train: 2775
Количество дубликатов в test: 1324


In [16]:
train = train[~train.duplicated()]
test = test[~test.duplicated()]

print(train.shape)
print(test.shape)

(35965, 2)
(24936, 2)


In [17]:
print(train.info())
print()
print(test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 35965 entries, 0 to 38739
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  35965 non-null  object
 1   text      35965 non-null  object
dtypes: object(2)
memory usage: 842.9+ KB
None

<class 'pandas.core.frame.DataFrame'>
Index: 24936 entries, 0 to 26259
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      24936 non-null  object
 1   category  24936 non-null  object
dtypes: object(2)
memory usage: 584.4+ KB
None


In [18]:
train['category'].value_counts()

,count
category,
autosport,3032
tennis,2933
martial_arts,2923
hockey,2919
football,2811
extreme,2786
volleyball,2780
winter_sport,2738
esport,2726


In [19]:
test['category'].value_counts()

,count
category,
winter_sport,2161
volleyball,2129
hockey,2120
athletics,2085
tennis,2002
autosport,1907
extreme,1866
esport,1855
motosport,1850


In [20]:
# Создадим объект LabelEncoder и переведем целевую переменную в числовой формат

le = LabelEncoder()

train['labels'] = le.fit_transform(train['category'])
test['labels'] = le.transform(test['category'])

# Удалим текстовую целевую переменную из данных

train = train.drop(columns=['category'])
test = test.drop(columns=['category'])

print(train.head(3))
print(test.head(3))

                                                text  labels
0  Волшебные фото Виктория Поплавская ЕвгенияМедв...      12
1  Возвращение в подземелье Треша 33 Эйфория тупо...       5
2  Лучшие чешские вратари – Доминик Доминатор Гаш...       6
                                                text  labels
0  СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...       4
1  Естественное восстановление после тяжелой трен...       8
2  Тема нарядов продолжается Одна из британских ж...      10


In [21]:
# Создадим словарь соответсвия для целевой переменной

label_names = le.classes_

names_dict = {}

for i in range(len(label_names)):
    names_dict[i] = label_names[i]

names_dict

{0: 'athletics',
 1: 'autosport',
 2: 'basketball',
 3: 'boardgames',
 4: 'esport',
 5: 'extreme',
 6: 'football',
 7: 'hockey',
 8: 'martial_arts',
 9: 'motosport',
 10: 'tennis',
 11: 'volleyball',
 12: 'winter_sport'}

### Modeling

In [22]:
# Загрузим модель и токенизатор

model_name = "cointegrated/rubert-tiny"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_names))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Переведем данные в необходимый для модели формат

train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

In [25]:
# Создадим функцию и токенизируем данные

def tokenize(batch):
    return tokenizer(batch['text'], max_length=128, padding='max_length', truncation=True)

train = train.map(tokenize, batched=True)
test = test.map(tokenize, batched=True)

Map:   0%|          | 0/35965 [00:00<?, ? examples/s]

Map:   0%|          | 0/24936 [00:00<?, ? examples/s]

In [26]:
# Функция для отображения метрик

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}


# Настроим отображение категорий

model.config.id2label = names_dict
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

In [27]:
# Аргументы обучения

args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

In [28]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-28-b311bde07710>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [29]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pavelzel95 (pavelzel95-self-employed) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.870900,1.357372,0.611285,0.601468
2,1.207700,1.085795,0.680101,0.680348
3,1.016000,1.024976,0.697145,0.698118


TrainOutput(global_step=6744, training_loss=1.364850353091637, metrics={'train_runtime': 9191.2431, 'train_samples_per_second': 11.739, 'train_steps_per_second': 0.734, 'total_flos': 199195313967360.0, 'train_loss': 1.364850353091637, 'epoch': 3.0})

In [30]:
model.save_pretrained("model/")
tokenizer.save_pretrained("model/")

('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.txt',
 'model/added_tokens.json',
 'model/tokenizer.json')

In [31]:
from transformers import pipeline

# Проверим работу внутри блокнота
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

result = classifier("Медведев закончил игру подачей навылет. Гейм, сет, матч")
print(result)


Device set to use cpu


[{'label': 'tennis', 'score': 0.9044870138168335}]


In [32]:
result = classifier("Отличный прыжок нашего спортсмена со снаряда, идеальное приземление")
print(result)

[{'label': 'athletics', 'score': 0.6101474165916443}]
